In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
import pickle
import pytorch_lightning as pl
import torchvision.transforms as T

from PIL import Image
from pathlib import Path
from pytorch_lightning.callbacks import TQDMProgressBar
from tqdm.notebook import tqdm
from typing import List, Union, Callable

from lib.datamodule import ImageNetModule, CatStudyModule
from lib.model import ImageNetModel

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# Create GradCAMs

This notebook can be used to create GradCAM visualizations using a classifier created with ```07_train_model_catstudy.ipynb```.

In [3]:
# set a model checkpoint
checkpoint = 'checkpoints/ImageNetModel_Pilot_epoch14_val_acc0.73.ckpt'

# load model 
model = ImageNetModel().load_from_checkpoint(checkpoint)

/old_home/ammeling/projects/ImageNet/.env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/old_home/ammeling/projects/ImageNet/.env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# set up grad cam
target_layers = [model.feature_extractor[-2]] # last conv layer before AdAvgPool

# construct cam object
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

In [5]:
# set grad cam dir 
grad_cam_dir = Path('image_files_GradCAM')
grad_cam_dir.mkdir(exist_ok=True, parents=True)

In [6]:
from skimage.transform import resize as numpy_resize

# patch size the model was trained with
size = 224

# set transforms
to_tensor = T.Compose([
    T.Resize((size, size)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
resize = T.Resize((size,size))

In [7]:
# # excluded data set 
# excluded_dir = Path('image_files/Excluded')

# # make dir
# sub_dir = grad_cam_dir.joinpath(excluded_dir.name)
# sub_dir.mkdir(exist_ok=True, parents=True)

# # select an image
# for idx, cat in enumerate(excluded_dir.iterdir()):

#     # load image 
#     rgb_img = Image.open(cat).convert('RGB')

#     # create transfrom to scale back to original size
#     width, height = rgb_img.size
#     resize_up = T.Resize((height, width))

#     # prepare image 
#     input_tensor = to_tensor(rgb_img)
#     rgb_img = np.array(rgb_img) / 255

#     # construct GradCAM
#     grayscale_cam = cam(input_tensor=input_tensor.unsqueeze(0))
#     grayscale_cam = grayscale_cam.squeeze() 
#     grayscale_cam = numpy_resize(grayscale_cam, (height, width))
#     visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

#     # scale GradCAM back to original size
#     visualization = Image.fromarray(visualization)
#     visualization = resize_up(visualization)

#     # save grad cam
#     save_name = sub_dir.joinpath(cat.name)
#     visualization.save(save_name)

In [8]:
# experimental study set 
experimental_dir = Path('image_files/Experimental_study_set')

# make dir
sub_dir = grad_cam_dir.joinpath(experimental_dir.name)
sub_dir.mkdir(exist_ok=True, parents=True)

# select an image
for idx, cat in enumerate(experimental_dir.iterdir()):

    # load image 
    rgb_img = Image.open(cat).convert('RGB')

    # create transfrom to scale back to original size
    width, height = rgb_img.size
    resize_up = T.Resize((height, width))

    # prepare image 
    input_tensor = to_tensor(rgb_img)
    rgb_img = np.array(rgb_img) / 255

    # construct GradCAM
    grayscale_cam = cam(input_tensor=input_tensor.unsqueeze(0))
    grayscale_cam = grayscale_cam.squeeze() 
    grayscale_cam = numpy_resize(grayscale_cam, (height, width))
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    # scale GradCAM back to original size
    visualization = Image.fromarray(visualization)
    visualization = resize_up(visualization)

    # save grad cam
    save_name = sub_dir.joinpath(cat.name)
    visualization.save(save_name)

In [9]:
# recommender test set 
validation_dir = Path('image_files/Recommender_test_set')

# make dir
sub_dir = grad_cam_dir.joinpath(validation_dir.name)
sub_dir.mkdir(exist_ok=True, parents=True)

# select an image
for idx, cat in enumerate(validation_dir.iterdir()):

    # load image 
    rgb_img = Image.open(cat).convert('RGB')

    # create transfrom to scale back to original size
    width, height = rgb_img.size
    resize_up = T.Resize((height, width))

    # prepare image 
    input_tensor = to_tensor(rgb_img)
    rgb_img = np.array(rgb_img) / 255

    # construct GradCAM
    grayscale_cam = cam(input_tensor=input_tensor.unsqueeze(0))
    grayscale_cam = grayscale_cam.squeeze() 
    grayscale_cam = numpy_resize(grayscale_cam, (height, width))
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    # scale GradCAM back to original size
    visualization = Image.fromarray(visualization)
    visualization = resize_up(visualization)

    # save grad cam
    save_name = sub_dir.joinpath(cat.name)
    visualization.save(save_name)